In [4]:
import numpy as np
import pandas as pd
import seaborn as sns

In [5]:
import warnings
warnings.filterwarnings('ignore')

# 1. Iris

In [116]:
iris = sns.load_dataset('iris')

## 1. 붓꽃 종(species)별로 꽃잎길이(sepal_length), 꽃잎폭(sepal_width), 꽃받침길이(petal_length), 꽃받침폭(petal_width)의 평균, 표준편차 등 기초통계량(describe())을 구하시오.

In [119]:
iris.groupby('species').agg(['mean', 'std'])

sepal_length           sepal_width           petal_length  \
                   mean       std        mean       std         mean   
species                                                                
setosa            5.006  0.352490       3.428  0.379064        1.462   
versicolor        5.936  0.516171       2.770  0.313798        4.260   
virginica         6.588  0.635880       2.974  0.322497        5.552   

                     petal_width            
                 std        mean       std  
species                                     
setosa      0.173664       0.246  0.105386  
versicolor  0.469911       1.326  0.197753  
virginica   0.551895       2.026  0.274650

In [117]:
# 1
iris.groupby(iris.species).describe().T

species                setosa  versicolor  virginica
sepal_length count  50.000000   50.000000  50.000000
             mean    5.006000    5.936000   6.588000
             std     0.352490    0.516171   0.635880
             min     4.300000    4.900000   4.900000
             25%     4.800000    5.600000   6.225000
             50%     5.000000    5.900000   6.500000
             75%     5.200000    6.300000   6.900000
             max     5.800000    7.000000   7.900000
sepal_width  count  50.000000   50.000000  50.000000
             mean    3.428000    2.770000   2.974000
             std     0.379064    0.313798   0.322497
             min     2.300000    2.000000   2.200000
             25%     3.200000    2.525000   2.800000
             50%     3.400000    2.800000   3.000000
             75%     3.675000    3.000000   3.175000
             max     4.400000    3.400000   3.800000
petal_length count  50.000000   50.000000  50.000000
             mean    1.462000    4.260000   5.552000
             std     0.173664    0.469911   0.551895
             min     1.000000    3.000000   4.500000
             25%     1.400000    4.000000   5.100000
             50%     1.500000    4.350000   5.550000
             75%     1.575000    4.600000   5.875000
             max     1.900000    5.100000   6.900000
petal_width  count  50.000000   50.000000  50.000000
             mean    0.246000    1.326000   2.026000
             std     0.105386    0.197753   0.274650
             min     0.100000    1.000000   1.400000
             25%     0.200000    1.200000   1.800000
             50%     0.200000    1.300000   2.000000
             75%     0.300000    1.500000   2.300000
             max     0.600000    1.800000   2.500000

## 2. 3분위수(Q3)와 1분위수(Q1)의 차이보다 1.5배가 크거나 작은 데이터는 이상치이다. <br>즉,<br>Q1 - 1.5 * (Q3 - Q1) 보다 작은 데이터<br>Q3 + 1.5 * (Q3 - Q1) 보다 큰 데이터<br>이 이상치를 제거하고 위의 4가지 항목에 대해서 평균, 표준편차를 구하시오.

In [120]:
s = iris[iris.species=='setosa']['sepal_length']
q1, q3 = s.quantile(.25), s.quantile(.75)
iqr = q3 - q1

q1, q3, iqr

(4.8, 5.2, 0.40000000000000036)

In [128]:
lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr
s[(s < lower) | (s > upper)] = np.nan
s.isnull().sum()
# 0 이 나오면 이상치 없음

0

In [129]:
s = iris[iris.species=='setosa']['sepal_width']
s.mean(), s.std()

(3.428000000000001, 0.3790643690962886)

In [130]:
q1, q3 = s.quantile(.25), s.quantile(.75)
iqr = q3 - q1

In [131]:
lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr
s[(s < lower) | (s > upper)] = np.nan
s.isnull().sum()

2

In [138]:
def get_new_stat(s):
    q1, q3 = s.quantile(.25), s.quantile(.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    s[(s < lower) | (s > upper)] = np.nan
    outlier = s.isnull().sum() != 0
    return outlier, np.round(s.mean(), 2), np.round(s.std(), 2)

In [145]:
species_list, feature_list, mean_list, std_list = [], [], [], []
outlier_list, new_mean_list, new_std_list = [], [], []
for species in iris.species.unique():

    for feature in iris.columns[:-1]:
        s = iris[iris.species == species][feature]
        species_list.append(species)
        feature_list.append(feature)
        mean_list.append(np.round(s.mean(), 2))
        std_list.append(np.round(s.std(), 4))

        outlier, new_mean, new_std = get_new_stat(s)
        outlier_list.append(outlier)
        new_mean_list.append(new_mean)
        new_std_list.append(new_std)

In [146]:
df = pd.DataFrame({
    'species' : species_list, 'feature' : feature_list,
    'mean' : mean_list, 'std' : std_list,
    'outlier' : outlier_list, 'new_mean' : new_mean_list, 'new_std' : new_std_list
})

df.set_index(['species', 'feature'], inplace = True)
df

mean     std  outlier  new_mean  new_std
species    feature                                               
setosa     sepal_length  5.01  0.3525    False      5.01     0.35
           sepal_width   3.43  0.3791     True      3.43     0.32
           petal_length  1.46  0.1737     True      1.46     0.13
           petal_width   0.25  0.1054     True      0.23     0.09
versicolor sepal_length  5.94  0.5162    False      5.94     0.52
           sepal_width   2.77  0.3138    False      2.77     0.31
           petal_length  4.26  0.4699     True      4.29     0.44
           petal_width   1.33  0.1978    False      1.33     0.20
virginica  sepal_length  6.59  0.6359     True      6.62     0.59
           sepal_width   2.97  0.3225     True      2.96     0.26
           petal_length  5.55  0.5519    False      5.55     0.55
           petal_width   2.03  0.2747    False      2.03     0.27

# 2. Titanic

In [27]:
titanic = sns.load_dataset('titanic')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## 1. 타이타닉호의 승객에 대해 나이와 성별에 의한 카테고리 열인 category1 열을 만드시오. category1 카테고리는 다음과 같이 정의됨
1. 20살이 넘으면 성별을 그대로 사용한다.
2. 20살 미만이면 성별에 관계없이 “child”라고 한다.

In [7]:
titanic['category1'] = titanic.apply(lambda x: x.sex if x.age >= 20 else 'child', axis = 1)
titanic.head(10)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,male
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,female
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,female
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,female
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,male
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True,child
6,0,1,male,54.0,0,0,51.8625,S,First,man,True,E,Southampton,no,True,male
7,0,3,male,2.0,3,1,21.0750,S,Third,child,False,NaN,Southampton,no,False,child
8,1,3,female,27.0,0,2,11.1333,S,Third,woman,False,NaN,Southampton,yes,False,female
9,1,2,female,14.0,1,0,30.0708,C,Second,child,False,NaN,Cherbourg,yes,False,child


## 2. 타이타닉호의 승객 중 나이를 명시하지 않은 고객은 나이를 명시한 고객의 평균 나이 값이 되도록 titanic 데이터프레임을 고치시오.

In [28]:
titanic.age.fillna(np.round(titanic.age.mean(), 2), inplace=True)
titanic.head(10)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
5,0,3,male,29.7,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True
6,0,1,male,54.0,0,0,51.8625,S,First,man,True,E,Southampton,no,True
7,0,3,male,2.0,3,1,21.0750,S,Third,child,False,NaN,Southampton,no,False
8,1,3,female,27.0,0,2,11.1333,S,Third,woman,False,NaN,Southampton,yes,False
9,1,2,female,14.0,1,0,30.0708,C,Second,child,False,NaN,Cherbourg,yes,False


## 3. 성별, 선실(class)별, 출발지(embark_town)별 생존율을 구하시오.

In [149]:
# 성별
titanic.groupby('sex')[['survived']].mean()

,survived
sex,
female,0.742038
male,0.188908


In [148]:
# 선실
titanic.pivot_table('survived', 'class')

,survived
class,
First,0.629630
Second,0.472826
Third,0.242363


In [150]:
# 출발지
titanic.groupby('embark_town')[['survived']].mean()

,survived
embark_town,
Cherbourg,0.553571
Queenstown,0.389610
Southampton,0.336957


In [161]:
titanic.pivot_table('survived', ['sex', 'class'])

survived
sex    class           
female First   0.968085
       Second  0.921053
       Third   0.500000
male   First   0.368852
       Second  0.157407
       Third   0.135447

In [160]:
# 세가지 동시
titanic.pivot_table('survived', ['sex', 'class'], 'embark_town')

embark_town    Cherbourg  Queenstown  Southampton
sex    class                                     
female First    0.976744    1.000000     0.958333
       Second   1.000000    1.000000     0.910448
       Third    0.652174    0.727273     0.375000
male   First    0.404762    0.000000     0.354430
       Second   0.200000    0.000000     0.154639
       Third    0.232558    0.076923     0.128302

## 4. 타이타닉호 승객을 ‘미성년자’, ‘청년’, ‘중년’, ‘장년’, ‘노년’ 나이 그룹으로 나누고, 각 그룹별 생존율을 구하시오.

In [151]:
bins = [1, 20, 30, 50, 70, 100]
labels = ["미성년자", "청년", "중년", "장년", "노년"]
titanic['age_group'] = pd.cut(titanic.age, bins, labels = labels)

titanic.groupby('age_group')['survived'].mean()

age_group
미성년자    0.424242
청년      0.334152
중년      0.423237
장년      0.355932
노년      0.200000
Name: survived, dtype: float64

## 5. qcut 명령으로 세 개의 나이 그룹을 만들고, 나이 그룹별 남녀 성비와 생존율을 구하시오

In [152]:
titanic['qcut_group'] = pd.qcut(titanic.age, 3, labels=["Age1", "Age2", "Age3"])
# 생존률
titanic.groupby('qcut_group')['survived'].mean()

qcut_group
Age1    0.411960
Age2    0.335526
Age3    0.405594
Name: survived, dtype: float64

In [162]:
# 남녀 성비
titanic.groupby('qcut_group')[['survived']].mean()

,survived
qcut_group,
Age1,0.411960
Age2,0.335526
Age3,0.405594


In [164]:
titanic['gender'] = titanic.apply(lambda r: 1 if r.sex == 'male' else 0, axis = 1)
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,age_group,qcut_group,gender
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,청년,Age2,1
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,미성년자,Age1,0
888,0,3,female,29.7,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,청년,Age2,0
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,청년,Age2,1
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,중년,Age3,1


In [165]:
titanic.pivot_table('gender', 'qcut_group')

,gender
qcut_group,
Age1,0.594684
Age2,0.680921
Age3,0.667832


# 3. MPG

In [6]:
mpg = sns.load_dataset('mpg')
mpg.head(10)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino
5,15.0,8,429.0,198.0,4341,10.0,70,usa,ford galaxie 500
6,14.0,8,454.0,220.0,4354,9.0,70,usa,chevrolet impala
7,14.0,8,440.0,215.0,4312,8.5,70,usa,plymouth fury iii
8,14.0,8,455.0,225.0,4425,10.0,70,usa,pontiac catalina
9,15.0,8,390.0,190.0,3850,8.5,70,usa,amc ambassador dpl


## 1. 배기량(displacement) 대비 마력(horsepower) 열(hp_per_cc)을 추가하시오.


In [15]:
# 1. 배기량(displacement) 대비 마력(horsepower) 열(hp_per_cc)을 추가하시오.
mpg['hp_per_cc'] = np.round(mpg.horsepower / mpg.displacement, 2)
mpg.tail()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,제조사,모델,hp_per_cc
393,27.0,4,140.0,86.0,2790,15.6,82,usa,ford,mustang gl,0.61
394,44.0,4,97.0,52.0,2130,24.6,82,europe,vw,pickup,0.54
395,32.0,4,135.0,84.0,2295,11.6,82,usa,dodge,rampage,0.62
396,28.0,4,120.0,79.0,2625,18.6,82,usa,ford,ranger,0.66
397,31.0,4,119.0,82.0,2720,19.4,82,usa,chevy,s-10,0.69


## 2. name으로부터 manufacturer(제조사)와 모델을 추출하여 새로운 열 manufacturer와 model을 추가하고, name 열은 삭제하시오.


In [156]:
manufacturer_list, model_list = [], []

for i in list(mpg.name.values):
    manufacturer = "".join(i.split()[:1])
    manufacturer_list.append(manufacturer)
    model = "".join(i.split()[1:])
    model_list.append(model)

mpg['manufacturer'] = manufacturer_list
mpg['model'] = model_list
del mpg['name']
mpg.tail(10)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,hp_per_cc,manufacturer,model
388,26.0,4,156.0,92.0,2585,14.5,82,usa,0.59,chrysler,lebaronmedallion
389,22.0,6,232.0,112.0,2835,14.7,82,usa,0.48,ford,granadal
390,32.0,4,144.0,96.0,2665,13.9,82,japan,0.67,toyota,celicagt
391,36.0,4,135.0,84.0,2370,13.0,82,usa,0.62,dodge,charger2.2
392,27.0,4,151.0,90.0,2950,17.3,82,usa,0.60,chevrolet,camaro
393,27.0,4,140.0,86.0,2790,15.6,82,usa,0.61,ford,mustanggl
394,44.0,4,97.0,52.0,2130,24.6,82,europe,0.54,vw,pickup
395,32.0,4,135.0,84.0,2295,11.6,82,usa,0.62,dodge,rampage
396,28.0,4,120.0,79.0,2625,18.6,82,usa,0.66,ford,ranger
397,31.0,4,119.0,82.0,2720,19.4,82,usa,0.69,chevy,s-10


In [9]:
mpg['제조사'] = mpg.name.apply(lambda x: x.split()[0])
mpg['모델'] = mpg.name.apply(lambda x: ' '.join(x.split()[1:]))
del mpg['name']
mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,제조사,모델
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet,chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick,skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth,satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc,rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford,torino


## 3. 엔진의 실린더(cylinders) 갯수별 연비(mpg)의 평균을 구하시오.


In [11]:
mpg.groupby('cylinders')[['mpg']].mean()


,mpg
cylinders,
3,20.550000
4,29.286765
5,27.366667
6,19.985714
8,14.963107


In [12]:
mpg.pivot_table('mpg', 'cylinders')

,mpg
cylinders,
3,20.550000
4,29.286765
5,27.366667
6,19.985714
8,14.963107


In [13]:
mpg[['mpg', 'cylinders']].groupby('cylinders').mean()

,mpg
cylinders,
3,20.550000
4,29.286765
5,27.366667
6,19.985714
8,14.963107


## 4. 생산지(origin)별 배기량 대비 마력(hp_per_cc)의 평균을 구하시오


In [16]:
mpg.groupby('origin')[['hp_per_cc']].mean()

,hp_per_cc
origin,
europe,0.739706
japan,0.789620
usa,0.511551


## 5. 모델이 5개 이상인 제조사에 대하여 연비(mpg)의 평균이 가장 좋은 제조사 Top 5를 구하시오.

In [19]:
mpg.제조사.value_counts()

ford             51
chevrolet        43
plymouth         31
amc              28
dodge            28
toyota           25
datsun           23
buick            17
pontiac          16
volkswagen       15
honda            13
mercury          11
mazda            10
oldsmobile       10
fiat              8
peugeot           8
audi              7
chrysler          6
vw                6
volvo             6
renault           5
saab              4
subaru            4
opel              4
chevy             3
bmw               2
cadillac          2
maxda             2
mercedes-benz     2
triumph           1
vokswagen         1
mercedes          1
hi                1
capri             1
chevroelt         1
toyouta           1
nissan            1
Name: 제조사, dtype: int64

In [20]:
s = mpg.제조사.value_counts()
s[s >= 5]

ford          51
chevrolet     43
plymouth      31
amc           28
dodge         28
toyota        25
datsun        23
buick         17
pontiac       16
volkswagen    15
honda         13
mercury       11
mazda         10
oldsmobile    10
fiat           8
peugeot        8
audi           7
chrysler       6
vw             6
volvo          6
renault        5
Name: 제조사, dtype: int64

In [21]:
s[s >= 5].index     # 아직은 리스트 x

Index(['ford', 'chevrolet', 'plymouth', 'amc', 'dodge', 'toyota', 'datsun',
       'buick', 'pontiac', 'volkswagen', 'honda', 'mercury', 'mazda',
       'oldsmobile', 'fiat', 'peugeot', 'audi', 'chrysler', 'vw', 'volvo',
       'renault'],
      dtype='object')

In [25]:
big = s[s >= 5].index.tolist()
print(big)

['ford', 'chevrolet', 'plymouth', 'amc', 'dodge', 'toyota', 'datsun', 'buick', 'pontiac', 'volkswagen', 'honda', 'mercury', 'mazda', 'oldsmobile', 'fiat', 'peugeot', 'audi', 'chrysler', 'vw', 'volvo', 'renault']


In [26]:
# in 연산자 대신에 isin() method 사용
mpg[mpg.제조사.isin(big)].head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,제조사,모델,hp_per_cc
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet,chevelle malibu,0.42
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick,skylark 320,0.47
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth,satellite,0.47
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc,rebel sst,0.49
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford,torino,0.46


In [30]:
big_mpg = mpg[mpg.제조사.isin(big)]
big_mpg.pivot_table('mpg', '제조사')

,mpg
제조사,
amc,18.246429
audi,26.714286
buick,19.182353
chevrolet,20.472093
chrysler,17.266667
datsun,31.113043
dodge,22.060714
fiat,28.912500
ford,19.694118


In [32]:
big_mpg.pivot_table('mpg', '제조사').sort_values(by= 'mpg', ascending = False).head()

,mpg
제조사,
vw,39.016667
honda,33.761538
renault,32.880000
datsun,31.113043
mazda,30.860000


### filter 사용

In [33]:
mpg.groupby('제조사').filter(lambda x: x.제조사.count() >= 5).head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,제조사,모델,hp_per_cc
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet,chevelle malibu,0.42
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick,skylark 320,0.47
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth,satellite,0.47
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc,rebel sst,0.49
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford,torino,0.46


In [34]:
new_mpg = mpg.groupby('제조사').filter(lambda x: x.제조사.count() >= 5)
new_mpg.pivot_table('mpg', '제조사').sort_values(by='mpg', ascending= False).head()

,mpg
제조사,
vw,39.016667
honda,33.761538
renault,32.880000
datsun,31.113043
mazda,30.860000
